In [1]:
!pip install datasets
import pandas as pd
from datasets import get_dataset_split_names, load_dataset, ClassLabel, Value
convo=load_dataset("allenai/prosocial-dialog")
convo

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Found cached dataset json (C:/Users/Saif/.cache/huggingface/datasets/allenai___json/allenai--prosocial-dialog-ebbad39ca08b6d44/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['context', 'response', 'rots', 'safety_label', 'safety_annotations', 'safety_annotation_reasons', 'source', 'etc', 'dialogue_id', 'response_id', 'episode_done'],
        num_rows: 120236
    })
    validation: Dataset({
        features: ['context', 'response', 'rots', 'safety_label', 'safety_annotations', 'safety_annotation_reasons', 'source', 'etc', 'dialogue_id', 'response_id', 'episode_done'],
        num_rows: 20416
    })
    test: Dataset({
        features: ['context', 'response', 'rots', 'safety_label', 'safety_annotations', 'safety_annotation_reasons', 'source', 'etc', 'dialogue_id', 'response_id', 'episode_done'],
        num_rows: 25029
    })
})

In [13]:
from transformers import   AutoModelForCausalLM, AutoTokenizer,DataCollatorForLanguageModeling
model_ckpt="transformersegmentation/GPT2-gpt2_lm_head_model-model"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model =  AutoModelForCausalLM.from_pretrained(model_ckpt)
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [25]:
tokenizer.pad_token = tokenizer.eos_token
def tokenize_text(data, padding=True):
  return tokenizer(data["response"],padding=True, truncation=True, max_length=128,)




In [26]:
convo  = convo.map(tokenize_text, batched=True,)

Map:   0%|          | 0/120236 [00:00<?, ? examples/s]

Map:   0%|          | 0/20416 [00:00<?, ? examples/s]

Map:   0%|          | 0/25029 [00:00<?, ? examples/s]

In [27]:
from transformers import TrainingArguments

batch_size=2

logging_steps =  len(convo["train"])
output_dir = "hopefullychat_bot"
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    per_gpu_eval_batch_size=batch_size,
    per_gpu_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    per_device_train_batch_size=batch_size,
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [28]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=convo["train"],
    eval_dataset=convo["validation"],
    tokenizer=tokenizer,
)


RuntimeError: false INTERNAL ASSERT FAILED at "..\\c10/cuda/CUDAGraphsC10Utils.h":74, please report a bug to PyTorch. Unknown CUDA graph CaptureStatus1537035536

In [12]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from huggingface_hub import notebook_login

notebook_login()

In [23]:
trainer.train()


C:\Users\Saif\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/45090 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [24]:
import torch
torch.cuda.empty_cache()


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.